# 네이버 영화정보 크롤링
- top_movie_data에 들어있는 영화들에 대한 정보 크롤링
- 'https://movie.naver.com/' 여기서 영화검색을 입력하고 그 영화의 url상 고유코드번호를 먼저 수집한다
    - ex) 
    - 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=69783'
- 수집한 고유코드를 영화제목에 대응시켜서 dataframe을 만든다
- 영화정보 url에 영화 고유코드만 바꿔가면서 scrapy로 크롤링한다
- 영화 평점리뷰도 페이지를 url로 바꿔서 크롤링 할 수 있다(마지막의 page 바꾸면 됨)
    - https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=44885&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=4
- 배우 
    - https://movie.naver.com/movie/bi/mi/detail.nhn?code=44885
    
- 크롤링 과정
    - top_movie_data에서 영화제목 컬럼만 가져와서, 셀레니움으로 영화제목을 검색란 input에 넣고 엔터친다음에 
    - webdriver.current_url로 현재 url의 영화제목 고유코드를 dataframe에 모은다
    - 이 dataframe을 완성한 이후에는 
    - `https://movie.naver.com/movie/bi/mi/basic.nhn?code={}'.format()` 이거로 영화제목의 고유코드로 scrapy를 이용해서 정보 크롤링
    
- movie_title_list[718] : '독전 - 익스텐디드 컷' -> 네이버 영화 검색결과가 없어서 그냥 '독전'으로 찾아서 넣음

In [83]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from selenium import webdriver
import re
import matplotlib as mpl
from matplotlib import rc
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

# matplotlib 한글 사용을 위한 코드
rc('font', family='AppleGothic')

In [84]:
top_movie_data = pd.read_csv('top_movie_data.csv')
top_movie_data.head(2)

,Unnamed: 0,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,1843,88582586366,10494499,25001906266,2823388,액션,12세이상관람가,일반영화
1,1,국제시장,윤제균,"(주)제이케이필름,씨제이이앤엠(주)",NaN,씨제이이앤엠(주),2014-12-17,개봉영화,장편,한국,966,69823893034,8911437,16331193634,2033863,드라마,12세이상관람가,일반영화


In [85]:
top_movie_data = top_movie_data.rename(columns={'Unnamed: 0':'id'})

In [91]:
top_movie_data.head(2)

,id,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,1843,88582586366,10494499,25001906266,2823388,액션,12세이상관람가,일반영화
1,1,국제시장,윤제균,"(주)제이케이필름,씨제이이앤엠(주)",NaN,씨제이이앤엠(주),2014-12-17,개봉영화,장편,한국,966,69823893034,8911437,16331193634,2033863,드라마,12세이상관람가,일반영화


In [97]:
movie_title_list = list(top_movie_data['영화명'])
len(movie_title_list)

928

In [98]:
url = 'https://movie.naver.com/'
driver = webdriver.Chrome()
driver.get(url)

In [120]:
datas = []

In [153]:
for i in range(719, len(movie_title_list)):
    input_element = driver.find_element_by_css_selector('#ipt_tx_srch')
    input_element.send_keys(movie_title_list[i])
    time.sleep(1)
    try:
        driver.find_element_by_css_selector('#jAutoMV > ul > li > a').click()
    except Exception:
        driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1) > a').click()
        
    splited_url = driver.current_url.split('code=')
    data = {'N_movie_code' : splited_url[1]}
    datas.append(data)
    

In [154]:
len(datas)

928

In [156]:
movie_title_list[927]

'100일 동안 100가지로 100퍼센트 행복찾기'

In [157]:
datas[927]

{'N_movie_code': '183377'}

In [160]:
code_df = pd.DataFrame(datas)

In [162]:
df = pd.concat([top_movie_data, code_df], axis=1)

In [172]:
df[:10]

,id,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분,N_movie_code
0,0,어벤져스: 에이지 오브 울트론,조스 웨던,NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-04-23,개봉영화,장편,미국,1843,88582586366,10494499,25001906266,2823388,액션,12세이상관람가,일반영화,98438
1,1,국제시장,윤제균,"(주)제이케이필름,씨제이이앤엠(주)",NaN,씨제이이앤엠(주),2014-12-17,개봉영화,장편,한국,966,69823893034,8911437,16331193634,2033863,드라마,12세이상관람가,일반영화,102875
2,2,킹스맨 : 시크릿 에이전트,매튜 본,NaN,이십세기폭스코리아(주),이십세기폭스코리아(주),2015-02-11,개봉영화,장편,미국,706,50333635795,6125154,16707645195,1968882,액션,청소년관람불가,일반영화,114249
3,3,쥬라기 월드,콜린 트레보로우,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2015-06-11,개봉영화,장편,미국,1290,40130812071,4565888,11510753371,1243034,액션,12세이상관람가,일반영화,67786
4,4,조선명탐정 : 사라진 놉의 딸,김석윤,청년필름(주),NaN,(주)쇼박스,2015-02-11,개봉영화,장편,한국,811,30456879428,3872015,6908501928,845522,사극,12세이상관람가,일반영화,123596
5,5,매드 맥스: 분노의 도로,조지 밀러,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),2015-05-14,개봉영화,장편,미국,799,32781268462,3828994,11373970562,1263672,액션,15세이상관람가,일반영화,77768
6,6,분노의 질주: 더 세븐,제임스 완,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2015-04-01,개봉영화,장편,미국,983,26449334694,3247558,7328474894,864171,액션,15세이상관람가,일반영화,109905
7,7,스물,이병헌,"(주)영화나무,(주)아이에이치큐",NaN,(주)넥스트엔터테인먼트월드(NEW),2015-03-25,개봉영화,장편,한국,926,23558988686,3044134,5990587486,740243,코미디,15세이상관람가,일반영화,123277
8,8,빅 히어로,"돈 홀,크리스 윌리암스",NaN,월트디즈니컴퍼니코리아(주),월트디즈니컴퍼니코리아(주),2015-01-21,개봉영화,장편,미국,825,21993119765,2801949,6512949165,772249,애니메이션,전체관람가,일반영화,109911
9,9,극비수사,곽경택,"(주)제이콘컴퍼니,(주)영화사 신세계",NaN,(주)쇼박스,2015-06-18,개봉영화,장편,한국,894,18338857652,2330813,4800234552,588252,드라마,15세이상관람가,일반영화,122120


In [173]:
# df.at[3, 'N_movie_code'] = '67786'

## Scrapy 